
# **Wild Fire Exploratory Analysis:**
by: ***Daniel Scurlock*** (3/2/20)


---



# *Dataset Description:*
https://www.kaggle.com/rtatman/188-million-us-wildfires

This data publication contains a spatial database of wildfires that occurred in the United States from 1992 to 2015. It is the third update of a publication originally generated to support the national Fire Program Analysis (FPA) system. The wildfire records were acquired from the reporting systems of federal, state, and local fire organizations. The following core data elements were required for records to be included in this data publication: discovery date, final fire size, and a point location at least as precise as Public Land Survey System (PLSS) section (1-square mile grid). The data were transformed to conform, when possible, to the data standards of the National Wildfire Coordinating Group (NWCG). Basic error-checking was performed and redundant records were identified and removed, to the degree possible. The resulting product, referred to as the Fire Program Analysis fire-occurrence database (FPA FOD), includes 1.88 million geo-referenced wildfire records, representing a total of 140 million acres burned during the 24-year period.


---


***I will try to be as thorough as possible. My plans going in are to explore, clean, understand, and explain. I hope that this notebook remains in accord with that frame of thought***

In [0]:

# This is an Sqlite database, and i do not want it to be so

import sqlite3
import pandas as pd
import io
import os
import datetime

###############################################################
# Function to read a table from Sqlite
# and convert it to csv, with columns 
# specified as a argument

def sqlite_to_csv(database, table, columns, output):
  
  # Connect to Database
  connection = sqlite3.connect(database)
  
  # Build the string for columns needed
  cols = ','.join(columns) 
  
  # Make an sql command from them
  sqlstr = 'SELECT '+cols+' FROM '+table 
  
  # Execute
  df = pd.read_sql_query(sqlstr, connection)

  df.columns = columns
  
  df.to_csv(output, header=True, encoding='utf-8')

  # Clean up
  connection.close()
  ###########################################################

# Getting what I need from sqlite into a CSV


In [0]:
# What do we need from the fires table?
# this may change as we go along, but
# for right now, all I need is these 

names = [
    "FIRE_NAME",
    "FIRE_YEAR",
    "DISCOVERY_DATE",
    "DISCOVERY_DOY",
    "DISCOVERY_TIME",
    "STAT_CAUSE_CODE",
    "CONT_DATE",
    "STAT_CAUSE_DESCR",
    "CONT_DOY",
    "CONT_TIME",
    "FIRE_SIZE",
    "FIRE_SIZE_CLASS",
    "LATITUDE",
    "LONGITUDE",
    "STATE",
    "COUNTY",
]

# Lets get it into CSV
sqlite_to_csv('/content/drive/My Drive/Lambda/DSU1-BUILD/data/FPA_FOD_20170508.sqlite', 
              'fires', 
              names,
              '/content/drive/My Drive/Lambda/DSU1-BUILD/data/fires.csv')

# Looking for things that need cleaning

In [0]:
# Read the new csv in
firesdf = pd.read_csv('/content/drive/My Drive/Lambda/DSU1-BUILD/data/fires.csv')

In [4]:
# Looking at the first few rows
firesdf.head()

,Unnamed: 0,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,CONT_DATE,STAT_CAUSE_DESCR,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY
0,0,FOUNTAIN,2005,2453403.5,33,1300.0,9.0,2453403.5,Miscellaneous,33.0,1730.0,0.10,A,40.036944,-121.005833,CA,63
1,1,PIGEON,2004,2453137.5,133,845.0,1.0,2453137.5,Lightning,133.0,1530.0,0.25,A,38.933056,-120.404444,CA,61
2,2,SLACK,2004,2453156.5,152,1921.0,5.0,2453156.5,Debris Burning,152.0,2024.0,0.10,A,38.984167,-120.735556,CA,17
3,3,DEER,2004,2453184.5,180,1600.0,1.0,2453189.5,Lightning,185.0,1400.0,0.10,A,38.559167,-119.913333,CA,3
4,4,STEVENOT,2004,2453184.5,180,1600.0,1.0,2453189.5,Lightning,185.0,1200.0,0.10,A,38.559167,-119.933056,CA,3


In [5]:
# Looking at the last ffew rows
firesdf.tail()

,Unnamed: 0,FIRE_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,CONT_DATE,STAT_CAUSE_DESCR,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY
1880460,1880460,ODESSA 2,2015,2457291.5,269,1726.0,13.0,2457291.5,Missing/Undefined,269.0,1843.0,0.01,A,40.481637,-122.389375,CA,NaN
1880461,1880461,NaN,2015,2457300.5,278,126.0,9.0,NaN,Miscellaneous,NaN,NaN,0.20,A,37.617619,-120.938570,CA,NaN
1880462,1880462,NaN,2015,2457144.5,122,2052.0,13.0,NaN,Missing/Undefined,NaN,NaN,0.10,A,37.617619,-120.938570,CA,NaN
1880463,1880463,NaN,2015,2457309.5,287,2309.0,13.0,NaN,Missing/Undefined,NaN,NaN,2.00,B,37.672235,-120.898356,CA,NaN
1880464,1880464,BARKER BL BIG_BEAR_LAKE_,2015,2457095.5,73,2128.0,9.0,NaN,Miscellaneous,NaN,NaN,0.10,A,34.263217,-116.830950,CA,NaN


In [6]:
# Nulls?
firesdf.isnull().sum()

Unnamed: 0               0
FIRE_NAME           960479
FIRE_YEAR                0
DISCOVERY_DATE           0
DISCOVERY_DOY            0
DISCOVERY_TIME      882638
STAT_CAUSE_CODE          0
CONT_DATE           891531
STAT_CAUSE_DESCR         0
CONT_DOY            891531
CONT_TIME           972553
FIRE_SIZE                0
FIRE_SIZE_CLASS          0
LATITUDE                 0
LONGITUDE                0
STATE                    0
COUNTY              678148
dtype: int64

In [0]:
# Way to many NaN values, I think I can live without
# most of these columns. I don't need a fire name. I dont
# really need a doscovery "time", since I have the date. 
# Conatainment information would be nice, but I am not 
# analyzing how long the fire lasted. A lot of missing for
# County, but we have long and lat. What I am after is 
# causation, demographics, times of the year when fires
# are worse....plus that unnamed col

firesdf = firesdf.drop(['Unnamed: 0',
                      'FIRE_NAME', 
                      'DISCOVERY_TIME', 
                      'CONT_DATE',
                      'CONT_DOY',
                      'CONT_TIME',
                      'COUNTY'], axis=1)

In [8]:
firesdf.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE
0,2005,2453403.5,33,9.0,Miscellaneous,0.10,A,40.036944,-121.005833,CA
1,2004,2453137.5,133,1.0,Lightning,0.25,A,38.933056,-120.404444,CA
2,2004,2453156.5,152,5.0,Debris Burning,0.10,A,38.984167,-120.735556,CA
3,2004,2453184.5,180,1.0,Lightning,0.10,A,38.559167,-119.913333,CA
4,2004,2453184.5,180,1.0,Lightning,0.10,A,38.559167,-119.933056,CA


In [9]:
firesdf.tail()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE
1880460,2015,2457291.5,269,13.0,Missing/Undefined,0.01,A,40.481637,-122.389375,CA
1880461,2015,2457300.5,278,9.0,Miscellaneous,0.20,A,37.617619,-120.938570,CA
1880462,2015,2457144.5,122,13.0,Missing/Undefined,0.10,A,37.617619,-120.938570,CA
1880463,2015,2457309.5,287,13.0,Missing/Undefined,2.00,B,37.672235,-120.898356,CA
1880464,2015,2457095.5,73,9.0,Miscellaneous,0.10,A,34.263217,-116.830950,CA


In [10]:
# What kind of Data is Pandas seeing?
firesdf.dtypes

FIRE_YEAR             int64
DISCOVERY_DATE      float64
DISCOVERY_DOY         int64
STAT_CAUSE_CODE     float64
STAT_CAUSE_DESCR     object
FIRE_SIZE           float64
FIRE_SIZE_CLASS      object
LATITUDE            float64
LONGITUDE           float64
STATE                object
dtype: object

In [0]:
# Can tell you right away, though i do not know why
# that our discovery date is Julian. Know nothing about
# Julian, other than what it is. I have Day of Year, and 
# year that I can use to get me a date

 
firesdf['DATE'] = pd.to_datetime(firesdf['FIRE_YEAR'], format='%Y') + pd.to_timedelta(firesdf['DISCOVERY_DOY'] - 1, unit='d')

In [12]:
firesdf.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DATE
0,2005,2453403.5,33,9.0,Miscellaneous,0.10,A,40.036944,-121.005833,CA,2005-02-02
1,2004,2453137.5,133,1.0,Lightning,0.25,A,38.933056,-120.404444,CA,2004-05-12
2,2004,2453156.5,152,5.0,Debris Burning,0.10,A,38.984167,-120.735556,CA,2004-05-31
3,2004,2453184.5,180,1.0,Lightning,0.10,A,38.559167,-119.913333,CA,2004-06-28
4,2004,2453184.5,180,1.0,Lightning,0.10,A,38.559167,-119.933056,CA,2004-06-28


In [13]:
firesdf.tail()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DATE
1880460,2015,2457291.5,269,13.0,Missing/Undefined,0.01,A,40.481637,-122.389375,CA,2015-09-26
1880461,2015,2457300.5,278,9.0,Miscellaneous,0.20,A,37.617619,-120.938570,CA,2015-10-05
1880462,2015,2457144.5,122,13.0,Missing/Undefined,0.10,A,37.617619,-120.938570,CA,2015-05-02
1880463,2015,2457309.5,287,13.0,Missing/Undefined,2.00,B,37.672235,-120.898356,CA,2015-10-14
1880464,2015,2457095.5,73,9.0,Miscellaneous,0.10,A,34.263217,-116.830950,CA,2015-03-14


In [0]:
# I can drop the other dates now, which just
# serve to annoy me, we can also remove the
# status code, as we have the descriptions

firesdf = firesdf.drop(['FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'STAT_CAUSE_CODE'], axis = 1)

In [15]:
firesdf.head()

,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,DATE
0,Miscellaneous,0.10,A,40.036944,-121.005833,CA,2005-02-02
1,Lightning,0.25,A,38.933056,-120.404444,CA,2004-05-12
2,Debris Burning,0.10,A,38.984167,-120.735556,CA,2004-05-31
3,Lightning,0.10,A,38.559167,-119.913333,CA,2004-06-28
4,Lightning,0.10,A,38.559167,-119.933056,CA,2004-06-28


In [0]:
# I think we can come up with better column names

firesdf.columns = ['Cause', 'Size', 'Class', 'Lat', 'Long', 'State', 'Date']

In [17]:
firesdf.head()

,Cause,Size,Class,Lat,Long,State,Date
0,Miscellaneous,0.10,A,40.036944,-121.005833,CA,2005-02-02
1,Lightning,0.25,A,38.933056,-120.404444,CA,2004-05-12
2,Debris Burning,0.10,A,38.984167,-120.735556,CA,2004-05-31
3,Lightning,0.10,A,38.559167,-119.913333,CA,2004-06-28
4,Lightning,0.10,A,38.559167,-119.933056,CA,2004-06-28


In [0]:
# I will have use of seperate features for years, months, and days

# Year
firesdf['Year'] = firesdf['Date'].dt.year
# Month Name
firesdf['Month'] = firesdf['Date'].dt.month_name()
# Day
firesdf['Day'] = firesdf['Date'].dt.day

In [19]:
firesdf.head()

,Cause,Size,Class,Lat,Long,State,Date,Year,Month,Day
0,Miscellaneous,0.10,A,40.036944,-121.005833,CA,2005-02-02,2005,February,2
1,Lightning,0.25,A,38.933056,-120.404444,CA,2004-05-12,2004,May,12
2,Debris Burning,0.10,A,38.984167,-120.735556,CA,2004-05-31,2004,May,31
3,Lightning,0.10,A,38.559167,-119.913333,CA,2004-06-28,2004,June,28
4,Lightning,0.10,A,38.559167,-119.933056,CA,2004-06-28,2004,June,28


In [0]:
# Change the order of things
firesfinal = firesdf[['Date', 'Year', 'Month', 'Day', 'State', 'Cause', 'Size', 'Class', 'Lat', 'Long']]


In [68]:
firesfinal.head()

,Date,Year,Month,Day,State,Cause,Size,Class,Lat,Long
0,2005-02-02,2005,February,2,CA,Miscellaneous,0.10,A,40.036944,-121.005833
1,2004-05-12,2004,May,12,CA,Lightning,0.25,A,38.933056,-120.404444
2,2004-05-31,2004,May,31,CA,Debris Burning,0.10,A,38.984167,-120.735556
3,2004-06-28,2004,June,28,CA,Lightning,0.10,A,38.559167,-119.913333
4,2004-06-28,2004,June,28,CA,Lightning,0.10,A,38.559167,-119.933056


In [69]:
firesfinal.tail()

,Date,Year,Month,Day,State,Cause,Size,Class,Lat,Long
1880460,2015-09-26,2015,September,26,CA,Missing/Undefined,0.01,A,40.481637,-122.389375
1880461,2015-10-05,2015,October,5,CA,Miscellaneous,0.20,A,37.617619,-120.938570
1880462,2015-05-02,2015,May,2,CA,Missing/Undefined,0.10,A,37.617619,-120.938570
1880463,2015-10-14,2015,October,14,CA,Missing/Undefined,2.00,B,37.672235,-120.898356
1880464,2015-03-14,2015,March,14,CA,Miscellaneous,0.10,A,34.263217,-116.830950


In [0]:
# Lets sort by year 

firesfinal = firesfinal.sort_values(by='Year')
firesfinal.reset_index(drop=True, inplace=True)


In [71]:
firesfinal.head()

,Date,Year,Month,Day,State,Cause,Size,Class,Lat,Long
0,1992-02-19,1992,February,19,MS,Arson,15.0,C,31.494948,-90.087916
1,1992-06-28,1992,June,28,WA,Lightning,1.0,B,48.616250,-117.990230
2,1992-06-28,1992,June,28,WA,Lightning,0.1,A,48.195880,-117.766750
3,1992-07-12,1992,July,12,WA,Arson,0.5,B,48.583490,-118.091330
4,1992-07-16,1992,July,16,WA,Children,0.1,A,48.705520,-118.008010


In [0]:
# Not what I would have preferd, but we have a decent 
# Dataframe to work with, and I have enough of a visual
# to proceed with the process of using this data to tell
# a story. So, writing the final to csv for use later. I had 
# planned on incoorperating weather data from anotehr API 
# into this dataframe (temp, wind speed, etc) for dates
# given the specific locatioon of the fire to test for any
# relationship between the two. There is only so much we
# can do with this particular dataset.

firesfinal.to_csv('/content/drive/My Drive/Lambda/DSU1-BUILD/data/fires_final.csv', header=True, encoding='utf-8')

In [0]:
# load final to see if all is well before tomorrow
dftest = pd.read_csv('/content/drive/My Drive/Lambda/DSU1-BUILD/data/fires_final.csv', index_col=0 )

In [78]:
dftest.head()

,Date,Year,Month,Day,State,Cause,Size,Class,Lat,Long
0,1992-02-19,1992,February,19,MS,Arson,15.0,C,31.494948,-90.087916
1,1992-06-28,1992,June,28,WA,Lightning,1.0,B,48.616250,-117.990230
2,1992-06-28,1992,June,28,WA,Lightning,0.1,A,48.195880,-117.766750
3,1992-07-12,1992,July,12,WA,Arson,0.5,B,48.583490,-118.091330
4,1992-07-16,1992,July,16,WA,Children,0.1,A,48.705520,-118.008010
